## 前置作業
在Anaconda Prompt中分別輸入:
- pip install mido
- pip install pygame

## 執行
依序執行所有cell，並打開MidiWrite視窗介面\
打開scores.xlsx檔，開始編寫歌曲
## 工作表數值存放位置
- 歌曲名稱: 工作表名稱
- BPM: A2
- 音軌數: A4
- 演奏樂器: A6、A7、A8
- 音符: B、D、F...
- 音符長度: C、E、G...

## 工作表編寫方式
- 歌曲名稱: 寫入歌曲的名稱，生成的.mid檔會以此命名
- BPM: 每分鐘的拍數，1拍=$\frac{60}{BPM}$秒
- 音軌數: 音樂的音軌數量
- 演奏樂器: 每個音軌要演奏的樂器
- 音符:
| 音符符號 | 編寫符號 |
| ----- | ----- |
| Do    | C / c |
| Re    | D / d |
| Mi    | E / e |
| Fa    | F / f |
| So    | G / g |
| La    | A / a |
| Si    | B / b|
| 升記號 | # |
| 降記號 | b |
| 八度音 | 0 ~ 8 |
| 休止符 | rest / REST / Rest |
- 音符長度: 音符的持續長度

In [62]:
import os
import tkinter as tk
import pygame.midi
from openpyxl import load_workbook
from mido import Message, MidiFile, MidiTrack

PATH = os.path.abspath('.')
NOTEXLSX = PATH + '\\scores.xlsx'
FONT =  'Arial'

In [63]:
class Database:
    def __init__(self):
        self.standard = {
            'C': 0,
            'C#': 1,
            'Db': 1,
            'D': 2,
            'D#': 3,
            'Eb': 3,
            'E': 4,
            'F': 5,
            'F#': 6,
            'Gb': 6,
            'G': 7,
            'G#': 8,
            'Ab': 8,
            'A': 9,
            'A#': 10,
            'Bb': 10,
            'B': 11
        }
        self.standard_lowercase = {
            'c': 0,
            'c#': 1,
            'db': 1,
            'd': 2,
            'd#': 3,
            'eb': 3,
            'e': 4,
            'f': 5,
            'f#': 6,
            'gb': 6,
            'g': 7,
            'g#': 8,
            'ab': 8,
            'a': 9,
            'a#': 10,
            'bb': 10,
            'b': 11
        }
        self.standard.update(self.standard_lowercase)
        
        self.rest = ['rest', 'Rest', 'REST']
        
        self.instruments = {
            'Acoustic Grand Piano': 0,     # 大鋼琴（聲學鋼琴）,平臺鋼琴
            'Bright Acoustic Piano': 1,    # 明亮的鋼琴
            'Electric Grand Piano': 2,     # 電鋼琴
            'Honky-tonk Piano': 3,         # 酒吧鋼琴
            'Rhodes Piano': 4,             # 柔和的電鋼琴
            'Chorused Piano': 5,           # 加合唱效果的電鋼琴
            'Harpsichord': 6,              # 羽管鍵琴（撥弦古鋼琴）,大鍵琴
            'Clavichord': 7,               # 科拉維科特琴（擊弦古鋼琴）
            'Celesta': 8,                  # 鋼片琴
            'Glockenspiel': 9,             # 鐘琴,鐵琴
            'Music box': 10,               # 八音盒,音樂盒
            'Vibraphone': 11,              # 顫音琴
            'Marimba': 12,                 # 馬林巴
            'Xylophone': 13,               # 木琴
            'Tubular Bells': 14,           # 管鐘
            'Dulcimer': 15,                # 大揚琴
            'Hammond Organ': 16,           # 擊桿風琴
            'Percussive Organ': 17,        # 打擊式風琴
            'Rock Organ': 18,              # 搖滾風琴
            'Church Organ': 19,            # 教堂風琴
            'Reed Organ': 20,              # 簧管風琴
            'Accordian': 21,               # 手風琴
            'Harmonica': 22,               # 口琴
            'Tango Accordian': 23,         # 探戈手風琴
            'Acoustic Guitar (nylon)': 24, # 尼龍弦吉他
            'Acoustic Guitar (steel)': 25, # 鋼弦吉他
            'Electric Guitar (jazz)': 26,  # 爵士電吉他
            'Electric Guitar (clean)': 27, # 清音電吉他
            'Electric Guitar (muted)': 28, # 悶音電吉他
            'Overdriven Guitar': 29,       # 加驅動效果的電吉他
            'Distortion Guitar': 30,       # 加失真效果的電吉他
            'Guitar Harmonics': 31,        # 吉他和音
            'Acoustic Bass': 32,           # 大貝司（聲學貝司）
            'Electric Bass (finger)': 33,  # 電貝司（指彈）
            'Electric Bass (pick)': 34,    # 電貝司（撥片）
            'Fretless Bass': 35,           # 無品貝司
            'Slap Bass 1': 36,             # 掌擊 Bass 1
            'Slap Bass 2': 37,             # 掌擊 Bass 2
            'Synth Bass 1': 38,            # 電子合成 Bass 1
            'Synth Bass 2': 39,            # 電子合成 Bass 2
            'Violin': 40,                  # 小提琴
            'Viola': 41,                   # 中提琴
            'Cello': 42,                   # 大提琴
            'Contrabass': 43,              # 低音大提琴
            'Tremolo Strings': 44,         # 弦樂群顫音音色
            'Pizzicato Strings': 45,       # 弦樂群撥弦音色
            'Orchestral Harp': 46,         # 豎琴
            'Timpani': 47,                 # 定音鼓
            'String Ensemble 1': 48,       # 弦樂合奏音色 1
            'String Ensemble 2': 49,       # 弦樂合奏音色 2
            'Synth Strings 1': 50,         # 合成弦樂合奏音色 1
            'Synth Strings 2': 51,         # 合成弦樂合奏音色 2
            'Choir Aahs': 52,              # 人聲合唱“啊”
            'Voice Oohs': 53,              # 人聲“嘟”
            'Synth Voice': 54,             # 合成人聲
            'Orchestra Hit': 55,           # 管弦樂敲擊齊奏
            'Trumpet': 56,                 # 小號
            'Trombone': 57,                # 長號
            'Tuba': 58,                    # 大號,低音號
            'Muted Trumpet': 59,           # 加弱音器小號
            'French Horn': 60,             # 法國號（圓號）
            'Brass Section': 61,           # 銅管組（銅管樂器合奏音色）
            'Synth Brass 1': 62,           # 合成銅管音色 1
            'Synth Brass 2': 63,           # 合成銅管音色 2
            'Soprano Sax': 64,             # 高音薩克斯風
            'Alto Sax': 65,                # 次中音薩克斯風
            'Tenor Sax': 66,               # 中音薩克斯風
            'Baritone Sax': 67,            # 低音薩克斯風
            'Oboe': 68,                    # 雙簧管
            'English Horn': 69,            # 英國管
            'Bassoon': 70,                 # 巴鬆（大管）
            'Clarinet': 71,                # 單簧管（黑管）
            'Piccolo': 72,                 # 短笛
            'Flute': 73,                   # 長笛
            'Recorder': 74,                # 木笛
            'Pan Flute': 75,               # 排簫
            'Bottle Blow': 76,             # 吹瓶子聲
            'Shakuhachi': 77,              # 日本尺八
            'Whistle': 78,                 # 口哨聲
            'Ocarina': 79,                 # 奧卡雷那,陶笛
            'Lead 1 (square)': 80,         # 合成主音 1 （方波）
            'Lead 2 (sawtooth)': 81,       # 合成主音 2 （鋸齒波）
            'Lead 3 (caliope lead)': 82,   # 蒸氣風琴
            'Lead 4 (chiff lead)': 83,     # 合成主音 4
            'Lead 5 (charang)': 84,        # 小型吉他
            'Lead 6 (voice)': 85,          # 合成主音 6 （人聲）
            'Lead 7 (fifths)': 86,         # 平行五度,第五鋸齒波
            'Lead 8 (bass+lead)': 87,      # 合成主音 8 （貝司加主音）,前衛低音
            'Pad 1 (new age)': 88,         # 合成音色 1 （新世紀）
            'Pad 2 (warm)': 89,            # 合成音色 2 （溫暖）
            'Pad 3 (polysynth)': 90,       # 合成音色 3,多型合成
            'Pad 4 (choir)': 91,           # 合成音色 4 （合唱）
            'Pad 5 (bowed)': 92,           # 合成音色 5,玻璃
            'Pad 6 (metallic)': 93,        # 合成音色 6 （金屬聲）
            'Pad 7 (halo)': 94,            # 合成音色 7 （光環）
            'Pad 8 (sweep)': 95,           # 合成音色 8,刮風
            'FX 1 (rain)': 96,             # 合成效果 1 雨聲
            'FX 2 (soundtrack)': 97,       # 合成效果 2 音軌,五度音
            'FX 3 (crystal)': 98,          # 合成效果 3 水晶
            'FX 4 (atmosphere)': 99,       # 合成效果 4 大氣
            'FX 5 (brightness)': 100,      # 合成效果 5 明亮
            'FX 6 (goblins)': 101,         # 合成效果 6 鬼怪
            'FX 7 (echoes)': 102,          # 合成效果 7 回聲
            'FX 8 (sci-fi)': 103,          # 合成效果 8 科幻
            'Sitar': 104,                  # 西塔爾（印度）
            'Banjo': 105,                  # 斑鳩琴（美洲）
            'Shamisen': 106,               # 三昧線（日本）
            'Koto': 107,                   # 十三弦箏（日本）
            'Kalimba': 108,                # 卡林巴
            'Bagpipe': 109,                # 風笛
            'Fiddle': 110,                 # 民族提琴
            'Shanai': 111,                 # 山奈
            'Tinkle Bell': 112,            # 叮當鈴
            'Agogo': 113,                  # 阿哥哥
            'Steel Drums': 114,            # 鋼鼓
            'Woodblock': 115,              # 木魚
            'Taiko Drum': 116,             # 太鼓
            'Melodic Tom': 117,            # 通通鼓,旋律中音鼓
            'Synth Drum': 118,             # 合成鼓
            'Reverse Cymbal': 119,         # 銅鈸,回音鈸
            'Guitar Fret Noise': 120,      # 吉他換把雜音
            'Breath Noise': 121,           # 呼吸聲
            'Seashore': 122,               # 海浪聲
            'Bird Tweet': 123,             # 鳥鳴
            'Telephone Ring': 124,         # 電話鈴
            'Helicopter': 125,             # 直升機
            'Applause': 126,               # 鼓掌聲
            'Gunshot': 127                 # 槍聲
        }

database = Database()

In [64]:
def note(notes, duration, track, channel, velocity = 100):
    def getNoteNum(note):
        name = note[:-1]
        octave = int(note[-1]) + 1
        return database.standard[name] + 12 * octave
        
    notes = notes.replace(' ', '').split(',')
    notes = [getNoteNum(note) for note in notes]
    
    for note in notes:
        track.append(Message("note_on", note = note, 
                             velocity = velocity, time = 0, 
                             channel = channel))
        
    for note in notes:
        if notes.index(note) > 0:
            duration = 0
        
        track.append(Message("note_off", note = note,
                             velocity = velocity, time = round(duration), 
                             channel = channel))
            
def rest(duration, track, channel):
    track.append(Message("note_off", time = round(duration), 
                         channel = channel))
    
def track(notesList, durationList, beat_time, instrument, track, channel):
    notes = [note.value for note in notesList if note.value != None]
    duration = [beat_time * float(beat.value) for beat in durationList if beat.value != None]
    
    track.append(Message("program_change", program = instrument,
                         channel = channel))

    for i in range(len(notes)):
        if notes[i] in database.rest:
            rest(duration[i], track, channel)
        else:
            note(notes[i], duration[i], track, channel)
            
    rest(beat_time * 1, track, channel)

In [65]:
def writeMidi(name, sheet, path):
    BPM = int(sheet['A2'].value)
    beat_time = 60 / BPM * 1000
    trackCount = int(sheet['A4'].value)
    
    midi = MidiFile()
    for i in range(trackCount):
        newTrack = MidiTrack()
        midi.tracks.append(newTrack)
        channel = i
        instrument = database.instruments[sheet['A' + str(6 + i)].value]
        track(sheet[chr(ord("B") + i * 2)], sheet[chr(ord("C") + i * 2)], 
              beat_time, instrument, newTrack, channel)
        
    midi.save(path)

In [66]:
def mixerInit():
    freq = 44100
    bitsize = -16
    channels = 2
    buffer = 1024
    pygame.mixer.init(freq, bitsize, channels, buffer)
    pygame.mixer.music.set_volume(1)
    
mixerInit()
    
def playMidi(file):
    try:
        pygame.mixer.music.load(file)
        pygame.mixer.music.play()
        return True
    except:
        return False
    
def stopMidi():
    pygame.mixer.music.stop()    

In [67]:
class window:
    def __init__(self):
        self.wb = load_workbook(NOTEXLSX)
        self.sheets = [sheet for sheet in self.wb]
        self.songs = [sheet.title for sheet in self.wb]
        self.filepaths = [PATH + '\music\\' + name + '.mid' for name in self.songs]
        self.songindex = 0

        self.window = tk.Tk()
        self.window.title('MidiWrite')
        self.window.geometry('500x600+1000+300')

        def reload():
            self.wb = load_workbook(NOTEXLSX)
            self.sheets = [sheet for sheet in self.wb]
            self.songs = [sheet.title for sheet in self.wb]
            self.filepaths = [PATH + '\music\\' + name + '.mid' for name in self.songs]
            self.songindex = min(self.songindex, len(self.songs) - 1)
            lab_songname.configure(text = self.songs[self.songindex])
            lab_hint.configure(text = f'讀取完畢\n{self.songs}\n')

        def last():
            self.songindex = (self.songindex - 1 + len(self.songs)) % len(self.songs)
            lab_songname.configure(text = self.songs[self.songindex])
            lab_hint.configure(text = f'上一首歌\n{self.songs[self.songindex]}\n')
            
        def next():
            self.songindex = (self.songindex + 1) % len(self.songs)
            lab_songname.configure(text = self.songs[self.songindex])
            lab_hint.configure(text = f'下一首歌\n{self.songs[self.songindex]}\n')

        def write():
            name = self.songs[self.songindex]
            sheet = self.sheets[self.songindex]
            filepath = self.filepaths[self.songindex]
            writeMidi(name, sheet, filepath)
            lab_hint.configure(text = f'寫入成功\n{name}:\n{filepath}\n')
            
        def play():
            name = self.songs[self.songindex]
            filepath = self.filepaths[self.songindex]
            success = playMidi(filepath)
            if success:
                lab_hint.configure(text = f'播放成功\n{name}:\n{filepath}\n')
            else:
                lab_hint.configure(text = f'播放失敗\n{name}:\n{filepath}\n')
            
        def stop():    
            stopMidi()
            lab_hint.configure(text = f'停止播放')
            
        lab_songname = tk.Label(self.window,
            text = self.songs[self.songindex], font = (FONT, 22))
        btn_reload = tk.Button(self.window,
            text = "重新讀取", font = (FONT, 16),
            command = reload)
        btn_last = tk.Button(self.window,
            text = "上一首", font = (FONT, 16),
            command = last)
        btn_next = tk.Button(self.window,
            text = "下一首", font = (FONT, 16),
            command = next)
        btn_write = tk.Button(self.window,
            text = "寫入midi", font = (FONT, 16),
            command = write)
        btn_play = tk.Button(self.window,
            text = "播放midi", font = (FONT, 16),
            command = play)
        btn_stop = tk.Button(self.window,
            text = "停止播放midi", font = (FONT, 16),
            command = stop)
        lab_hint = tk.Label(self.window,
            text = "", font = (FONT, 22))

        lab_songname.pack()
        btn_reload.pack()
        btn_last.pack()
        btn_next.pack()
        btn_write.pack()
        btn_play.pack()
        btn_stop.pack()
        lab_hint.pack()
        
        self.window.mainloop()

In [ ]:
window = window()